In [ ]:
!pip install tokenizers==0.10.2 transformers==4.6.0


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
  
tokenizer = AutoTokenizer.from_pretrained("Norod78/hebrew-gpt_neo-small")
model = AutoModelForCausalLM.from_pretrained("Norod78/hebrew-gpt_neo-small", pad_token_id=tokenizer.eos_token_id)

prompt_text = "אני אוהב שוקולד ועוגות"
max_len = 512
sample_output_num = 3
seed = 1000

import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = 0 if torch.cuda.is_available()==False else torch.cuda.device_count()

print(f"device: {device}, n_gpu: {n_gpu}")

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


device: cpu, n_gpu: 0


In [3]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

try:
    test_text = "שלום!"
    inputs = tokenizer(test_text, return_tensors="pt", padding=True, truncation=True)
    print("Tokenization successful!")
    print(inputs)
except Exception as e:
    print(f"Error during tokenization: {e}")
# הגדרת פונקציה להפקת embeddings
def get_embedding2(text):
    # המרת הטקסט ל-input_ids
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # הפעלת המודל להפקת embeddings
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # קבלת hidden states של כל הטוקנים
    hidden_states = outputs.hidden_states[-1]  # בוחרים את השכבה האחרונה

    # חישוב הממוצע של ה-embeddings של כל הטוקנים
    embeddings = hidden_states.mean(dim=1).squeeze().numpy()

    return embeddings

# בדיקת הפונקציה עם טקסט לדוגמה
input_text = "שלום! מה שלומך היום?"
embedding = get_embedding2(input_text)
print(embedding)

Tokenization successful!
{'input_ids': tensor([[3347,    5]]), 'attention_mask': tensor([[1, 1]])}
[-5.31943130e+00 -2.39071918e+00 -4.87468183e-01  6.97738230e-01
  1.66281432e-01  2.08134818e+00 -1.26299667e+00  2.44468451e-02
  1.65020788e+00  1.33752704e+00  3.25200617e-01  5.28960824e-01
 -7.92512417e-01 -3.71505111e-01  4.33558464e-01 -2.23307505e-01
  5.21111488e-01  4.62653100e-01  4.69726712e-01  7.19827235e-01
 -3.16448957e-01  9.00684774e-01 -2.61784554e-01 -1.55111563e+00
 -2.32391024e+00  4.21774656e-01  2.47925711e+00  6.79238319e-01
 -1.02987421e+00 -3.81442644e-02 -1.15482771e+00  2.54340577e+00
 -4.23125029e+00  6.83025360e-01  1.72842905e-01  3.16698343e-01
 -1.23741971e-02 -1.72327018e+00  2.00868893e+00 -8.14791560e-01
  4.53600359e+00 -3.86856019e-01  2.03044820e+00  2.74076915e+00
  1.03219807e+00 -2.15183330e+00  1.35063767e-01  2.76538539e+00
  2.07499242e+00  2.06412585e-03 -5.14078140e-02 -2.49391055e+00
 -7.45885551e-01  2.14131594e+00  4.35734361e-01  7.9932

In [4]:
import numpy as np
import json
#from google.colab import drive
#drive.mount('/content/drive')
# קריאת הקובץ JSON
with open('sample_data/פרי גני שוע.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# שליפת כל השאלות מהקובץ
questions = []
for book in data['contain'].values():
    for chapter in book.values():
        for verse in chapter.values():
            for qa in verse:
                questions.append(qa['question'])

In [5]:
# נניח שזה החלק הראשון שלך - חישוב embeddings
embeddings_filename = "save_embded/embeddings_step1.pkl"

# בדוק אם הנתונים כבר נשמרו, אם לא - הרץ ושמור
embeddings2 = load_from_pickle(embeddings_filename)
if embeddings2 is None:
    # הקוד שלך לחישוב ה-embeddings
    embeddings = get_embedding2(questions).numpy()
  # הפונקציה שלך
    save_to_pickle(embeddings2, embeddings_filename)


File save_embded/embeddings_step1.pkl not found!


: 

In [9]:
import faiss
# המרת כל השאלות ל-embeddings
# שלב נוסף - בניית FAISS index
index_filename = "save_embded/index_step2.pkl"

index2 = load_from_pickle(index_filename)
if index2 is None:
    # יצירת האינדקס אם הוא לא נטען מקובץ
    dimension2 = embeddings2.shape[1]
    index2 = faiss.IndexFlatL2(dimension2)
    index2.add(embeddings2)
    save_to_pickle(index2, index_filename)

# בניית אינדקס FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

: 

In [ ]:
import pickle

# שמירת ה-embeddings, השאלות והאינדקס
with open('save_embded/embeddings_and_index.pkl', 'wb') as f:
    pickle.dump({
        'embeddings': embeddings,
        'questions': questions,
        'faiss_index': index
    }, f)

print("ה-embeddings, השאלות והאינדקס נשמרו בהצלחה!")


In [1]:
import pickle
import os

def save_to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {filename}")

def load_from_pickle(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        print(f"Data loaded from {filename}")
        return data
    else:
        print(f"File {filename} not found!")
        return None



In [4]:
#טעינת קובץ ממקום קבוע
import os
import pickle

# נתיב הקובץ בפרויקט שלך
file_path = os.path.join(os.getcwd(), "save_embded/embeddings1.pkl")

# בדיקה אם הקובץ קיים
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        embeddings2 = pickle.load(f)
    print("ה-embeddings נטענו בהצלחה!")
else:
    print(f"הקובץ לא נמצא בנתיב: {file_path}")


ה-embeddings נטענו בהצלחה!


In [5]:
# import numpy as np
# import faiss

# # הנחת שהנתונים הם כבר בפורמט numpy array
# # אם לא, תשתמש ב np.array כדי להמיר אותם
# embeddings2 = np.array(embeddings2).astype('float32')

# בדוק את הממדים של ה-embeddings
print("Dimensions of embeddings2:", embeddings2.shape)

dimension = embeddings2.shape[1]
# index2 = faiss.IndexFlatL2(dimension2)
# print("FAISS index:", index2)

# # הוספת הנתונים לאינדקס
# index2.add(embeddings2)

# פונקציה למציאת השאלות הקרובות ביותר
def find_nearest_questions2(query, k=5):
    query_embedding = get_embedding2(query).astype('float32').reshape(1, -1)
    print("Query embedding dimensions:", query_embedding.shape)
    
    # בדוק אם ממדי ה-query תואמים את ממדי האינדקס
    if query_embedding.shape[1] != dimension2:
        raise ValueError(f"Dimension mismatch: query embedding dimension {query_embedding.shape[1]} does not match index dimension {dimension}")
    
    distances, indices = index.search(query_embedding, k)
    nearest_questions = [questions[idx] for idx in indices[0]]
    return nearest_questions

# שאלה לדוגמה
query = "האם נשים חיבות בתפילה"

# מציאת השאלות הקרובות ביותר
nearest_questions = find_nearest_questions2(query)
print("השאלות הקרובות ביותר לשאלה שלך הן:")
for q in nearest_questions:
    print(q)


Dimensions of embeddings2: (5158, 4096)
FAISS index: <faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001D871672EE0> >
Query embedding dimensions: (1, 768)


ValueError: Dimension mismatch: query embedding dimension 768 does not match index dimension 4096